<a href="https://colab.research.google.com/github/johannes-kk/am207/blob/master/exercises/10_variational_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAY 10: Variational Inference

### AM207: Advanced Scientific Computing

#### Instructor: Weiwei Pan

#### Due: October 6th, 11:59pm EST

**Names of Group Members**: Ansuman Prusty (ansumanprusty@gmail.com), Rob Clark roc4441@g.harvard.edu, Johannes Kolberg, johanneskolberg@g.harvard.edu, Yixuan Di, yixuan_di@g.harvard.edu, Gael Ancel gael_ancel@g.harvard.edu ___________

## Learning Goals:

1. Understand the similarities and differences between EM and CAVI
2. Understand, in broad strokes, the key assumptions that make these algorithms tractable for specific latent variable models
3. Understand the practicality/applicability of EM and CAVI


### Load necessary libraries

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures
from autograd import scipy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
%matplotlib inline

# Problem 1: Understanding EM and Variational Inference

In this problem, we will draw concrete connections between EM and variational inference by applying both methods to a certain class of latent variable models. You'll need to refer to relevant lecture notes on the derivations of EM and the derivation of the variational inference objective. This is an essay question that requires you to engage with complex derivations at a productive but still high level. No implementation is required.

#### **Non-Bayesian Latent Variable Model**
Recall the class of latent variable models we studied in lecture:
<img src="https://i.ibb.co/rvxGKC1/graphical-model.jpg" style="height:150px;">

#### **Bayesian Latent Variable Model**
A Bayesian version of the same class of models involve adding priors for the model parameters:
<img src="https://i.ibb.co/nzG6326/bayesian-model.jpg" style="height:150px;">


**Exercise 1:** **(Comparing ELBOs)** For the above type of Bayesian latent variable model, write down the ELBO for variational inference with a mean field variational family. Compare the variational inference ELBO for the Bayesian model to the expectation maximization ELBO for the non-Bayesian model. What are the differences and similarities between these two ELBOs?

  In both EM and variational inference we optimize the ELBO. Compare the update steps in EM to the update steps in Coordinate Ascent Variational Inference, draw a concrete analogy between them.
  
  ***Hint:*** To make both ELBO's comparable, make sure that both are in terms of $z, y, \theta, \phi$.

**Answer**<br>

## Expectation Maximization ELBO vs. Variational Inference ELBO (CAVI Optimization)

EM: $ELBO(\theta,\phi,z,y) = \mathbb{E}_{z\sim q(z)} \left[ \log\,\left(\frac{p(y, z|\theta, \phi)}{q(z)}\right)\right] = \mathbb{E}_{z\sim q(z)} \left[ \log\,\left(\frac{p(y|z,\theta)p(z|\phi)}{q(z)}\right)\right]$<br>
where $y = [y_1,...,y_n]$ and $z = [z_1...,z_n]$<br><br>

EM Algorithm: This algorithm consists in two steps, E-Step and M-Step. For the first we find a new function $q_{new}$ by maximizing the ELBO according to $q$ and in the M-Step we compute $\theta_{new}$ and $\phi_{new}$ by maximizing the ELBO according to $\theta$ and $\phi$. 

Bayesian VI ELBO: $ELBO(\theta,\phi,z,y) = \mathbb{E}_{\psi \sim q(\psi|\lambda)}\left[\log\left(\frac{p(\psi, y)}{q(\psi | \lambda)} \right)\right] = \mathbb{E}_{\phi,\theta,z \sim q(\phi,\theta,z |\lambda)}\left[\log\left(\frac{p(\phi,\theta,z , y)}{q(\phi,\theta,z  | \lambda)} \right)\right] = \mathbb{E}_{\phi,\theta,z \sim q(\phi,\theta,z |\lambda)}\left[\log\left(\frac{p(y|z,\phi)p(z|\theta)p(\phi)p(\theta)}{q(\phi| \lambda)q(\theta| \lambda)q(z| \lambda)} \right)\right]$
where $y = [y_1,...,y_n]$ and $z = [z_1...,z_n]$<br><br>

CAVI Algorithm: This new ELBO is computed with the the new target posterior distribution $p(\psi | Y_1,\dots,Y_N)$. To find its maximum we cannot use the gradient of this ELBO because the expression is too hard to compute. We will therefore iteratively maximize it over each $\lambda_i$, to find updated values of $\lambda_i$ which will be equal to $\text{arg max }_{\lambda} ELBO(\lambda_i,\lambda_{-i})$

We can however see that both methods maximize an objective function ELBO. One with prior one without. This is clear when we split out the joint distributions. The numerator of the ELBOs is split out with laws of probability  and we can see Bayesian VI ELBO includes a few more terms due to recognition of priors. Similarly in the denominator of the Bayesian VI ELBO, we see $q$ can be factorized (permitted by mean-field assumption) which also leaves us with a few more terms than EM for our priors.




**Exercise 2:** **(Comparing ELBOs and KL-divergences)** <br>
**Question**
Recall that the original objective of variational inference is to minimize a KL-divergence, we rewrote the objective to be that of maximizing the ELBO. Why is directly minimizing the KL-divergence in the original objective difficult (be specific about wherein the difficulty lies)? 

**Answer**
The KL divergence calculates the "distance" between a distribution we are comfortable, $q$, with and an intractable distribution, posterior $p(\psi|y)$, we hope to approximate. The intractable posterior in the denominator of the KL divergence cannot be evaluated. We need to get this KL divergence function in a form that involves distributions that are not intractable. With some transformations using properties of log and expectations, we can get an equation involving the ELBO. $p(\psi,y) = p(\phi,\theta,z,y) = p(y|z,\phi)p(z|\theta)p(\phi)p(\theta)$ in the numerator of the ELBO is an equation that we know how to evaluate since it is given in the model specification.


**Question**
  In the derivation of the E-step of EM, we reframed an maximization of the ELBO problem as a minimization of a KL-divergence problem. In this case, why was the KL-divergence easier to minimize and the ELBO harder to maximize (use the instantiation of the E-step for Gaussian Mixture Models in Lecture 7 to help support your answer)? 

**Answer**
For starters, it is optimization over a probability distribution, which is an uncomfortable task. The KL divergence is very easy to minimize because it can be proven that it is never < 0 and it is only equal to 0 when the two distributions we are calculating the "distance" between are exactly equal. So if we are maximizing an equation w.r.t. a distribution, it is definately worthwhile to see if the equation can be reformulated in terms of a KL divergence.

**Question**
  In the notes for Lecture 8, we introduce a way to maximize the variational inference ELBO -- through coordinate ascent. In the derivation of the updates for coordinate ascent, there is a place where we reframed an maximization of the ELBO problem as an equivalent minimization of a KL-divergence problem. Write down the exact form of this equivalence (the two expressions are separated in the derivation by a bunch of lines, you'll need to identify both parts that you need). In this case, why was the KL-divergence easier to minimize and the ELBO harder to maximization (use the instantiation of the update for Gaussian Mixture Models in Lecture 8 to help support your answer)?

**Question**
  Based on this analysis, can you draw some general conclusions about when we'd prefer to minimize the KL-divergence versus when we'd prefer to maximize the ELBO?

**Answer**
We want to mnimize KL-divergence rather than maximizing the ELBO when we are optimizing with respect to probability distributions.

**Exercise 3:** **(The Mean Field Assumption and Coordinate Ascent)**  Describe exactly when and how the mean field assumption is used in the derivation of the coordinate ascent updates.

**Answer**
The mean field assumption is what allows us to split up problem into components, which allows us to use coordinate ascent in the first place. It allows us to write joint variational distribution over all parameters as a product of the variational distribution over each individual parameter.

In terms of the math that gets us to coordinate ascent, it allows us to use fibini's theorem and independance of probability distributions to split out the ELBO as an iterated expectation.



**Exercise 4:** **(Generalizability of CAVI)** Summarize what kind of derivations/math is needed in order instantiate Coordinate Ascent Variational Inference (CAVI) for a given new model (look at what we did for Gaussian Mixture Models in Lecture 10 and predict what you'd need to do for a new model). Based on this, discuss the potential draw backs of using CAVI for Bayesian inference in general. Do these draw backs mean that variational inference is not a practical method of inference? What problem(s) need to be solved in order to make variational inference easy to implement for any given Bayesian model?

**Answer**
We end up with this CAVI update rule:

$$
q(\phi_i|\lambda_i^{new}) \propto \exp\left\{\mathbb{E}_{\psi_{-i} \sim q(\psi_{-i}|\lambda_{-i})} \left[\log \left(p(Y_1, \ldots, Y_N, \psi) \right)\right]\right\}
$$

This is the general solution, but unfortunately it is not directly useful to us. We must specify what this equation means in terms of parameters, hyperparameters, and data that we can plug in for each iteration of coordinate ascent. This means that we have to calculate it sepearately for each parameter in the model. As we've seen in the GMM with two parameters, $\mu$ and $z$, there were about 100 lines of math to get to a concrete formula we could use for these two parameter distributions.

This does not mean variational inference is an impractical method. We've traded computational inefficiency of MCMC for more math. This is often times the lesser of the two evils.

Our problem is that the optimization relies on the form of each parameter distribution. To make it more generalizable, we would need to find a variational distribution that can still approximate the posterior, but does not rely so much on the model specification. This seems much easier said than done.


**Exercise 5:** **(Generalizability of EM)** Summarize what kind of derivations/math is needed in order instantiate Expectation Maximization (EM) for a given new model (look at what we did for Gaussian Mixture Models in Lecture 9 and predict what you'd need to do for a new model). Based on this, discuss the potential draw backs of using EM for MLE inference in general. Do these draw backs mean that EM is not a practical method of inference? What problem(s) need to be solved in order to make EM easy to implement for any given latent variable model?

EM Models depend upon the E step for an estimate and M step for the update. Both E and M steps will ensure a model work if it converges. There is a chance that EM might not converge for some models. Also, EM models have the limitation of finding local minimas. So they may not work for all models.
